# Weigthed max-sat example

## Problem statement
The maximum satisfiability problem (MAX-SAT) is the problem of determining the maximum
number of clauses, of a given Boolean formula in conjunctive normal form, that can be made
true by an assignment of truth values to the variables of the formula.
One can define a weighted version of MAX-SAT as follows: given a conjunctive normal
form formula with weights assigned to each clause, find truth values for its variables
that maximize the combined weight of the satisfied clauses.
The MAX-SAT problem is an instance of weighted MAX-SAT where all weights are 1.

## Integer Linear Programming Formulation
MAX-SAT can be expressed using a binary linear program.
Fix a conjunctive normal form formula $F$ with variables $x_1, x_2, ..., x_n$,
and let $C$ denote the clauses of $F$.
For each clause $c$ in $C$, let $S^+_c$ and $S^−_c$ denote the sets of variables
which are not negated in $c$, and those that are negated in $c$, respectively.
The variables $y_x$ of the formulation will correspond to the variables of the formula $F$,
whereas the variables $z_c$ will correspond to the clauses.
The formulation is as follows:

maximize $\sum_{c \in C} w_c\cdot z_c$
(maximize the weight of the satisfied clauses)

subject to

$ z_c\leq\sum_{x\in S_c^+} y_x+\sum_{x\in S_c^-} (1-y_x)$
for all $c\in C$ (clause is true if and only if it has a true, non-negated variable or a false, negated one)

$z_c \in \{0,1\}$
for all $c\in C$
(every clause is either satisfied or not)

$y_x \in \{0,1\}$
for all $x\in F$
(every variable is either true or false)

## Particular Problem Instance (Weighted MAX-SAT)

$\phi = ({x_1} \vee \overline{x_2} : 3) \wedge (x_3 : 1) \wedge (\overline{x_3} \vee x_2 : 4)) $

## Formulate equivalent objective function using plain Python

In [ ]:
def f(x):
    x1, x2, x3 = x
    val = 0

    # clause 1
    if x1 or not x2:
        val += 3
    # clause 2
    if x3:
        val += 1
    # clause 3
    if not x3 or x2:
        val += 4

    return val

## Using AutoQUBO to convert to QUBO

In [11]:
# Install packages in the local env
!setup.py install

# Import everything
from autoqubo import SamplingCompiler, Utils
from itertools import product

# Create QUBO using the sampling compiler
# The solution is represetned by 3 bits
qubo, offset = SamplingCompiler.generate_qubo_matrix(fitness_function=f, input_size=3)

'setup.py' is not recognized as an internal or external command,
operable program or batch file.


## Print QUBO matrix and coefficients

In [ ]:
print("QUBO matrix:")
print(qubo)

print("QUBO coefficients")
print(f"x[] = {offset}")
for key, coefficient in Utils.get_matrix_dict_repr(qubo).items():
    i, j = key
    if i == j:
        print(f"x[{i+1}] = {coefficient}")
    else:
        print(f"x[{i+1}, {j+1}] = {coefficient}")


## Minimize using QUBO solver

In [ ]:
solutions, energy_values = Utils.solve(qubo, offset)
for s, e in zip(solutions, energy_values):
    print(f"x={s}, energy={e}")